# Image classification
In this notebook we are solving an image classification problem using Convolutional neural net. 

The data we are going to work with is the MNIST dataset which contains of 70000 numbers evenly distributed between 0-9.

The aim is to introduce the concept of convolutional layers and show its potential. The user should be familar with neural nets. This notebook is very similar to a notebook solving the problem with a traditional densly coupled neural net.

Some efforts has been made to show how the convolutions and pooling layers are working together. 

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

from scipy import ndimage
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils.multiclass import unique_labels

from skimage import measure

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Run twice to close the annoying warning created by numpy and tensorflow

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues,
                          fig_size=10):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]*100
    else:
        cm = cm
    

    fig, ax = plt.subplots(figsize=(fig_size,fig_size))
    im = ax.imshow(cm,norm=LogNorm(), cmap=cmap,
                interpolation='nearest')
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ylim=ax.get_ylim()
    ax.set(
        ylim=ylim,
        xticks=np.arange(cm.shape[1]),
        yticks=np.arange(cm.shape[0]),
        # ... and label them with the respective list entries
        xticklabels=classes, 
        yticklabels=classes,
        title=title,
        ylabel='True label',
        xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
              rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.1f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

def plot_errors(x_test, y_test, output,n_max=600):
    """ Function the reporting in a script in order to
    breake the script if it is estimated to take to long time"""
    n_not_corr = np.sum(output != y_test )
    n = int(np.ceil(np.sqrt(n_not_corr)))
    j = 0
    if n_not_corr > n_max:
        print('more then '+str(n_max),n**2)
        return
    f, ax = plt.subplots(n, n, figsize=(25, 25))
    ax = ax.flatten()
    
    for i in range(np.shape(output)[0]):
            if output[i]!=y_test[i]:
                ax[j].set_title(str(y_test[i]) + ' as ' + str(output[i]))
                ax[j].imshow(x_test[i,:,:,0], cmap='gray')
                ax[j].axis('off')
                j+=1
    for x in ax.ravel():
        x.axis("off")
    plt.subplots_adjust(bottom=-0.09, wspace=0.03)
    plt.show()

### Load the dataset
We also need to take a first look of the dataset by looking at the shape which gives us an idaea of the structure and size. We also show the dataset distribution.

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print('Shape before reshape:',np.shape(x_train))
x_train = x_train.reshape(-1,28, 28, 1)   #Reshape for CNN !!
x_test = x_test.reshape(-1,28, 28, 1)     #The added dimension is to account for RRG images

##
print('Train data:',np.shape(x_train))
print('Test data:', np.shape(x_test))

y_count = np.bincount(y_train)
ii = np.nonzero(y_count)[0]
plt.bar(ii, y_count)
plt.xticks(ii)
plt.title('Distribution of numbers')
plt.show()


# Look at the dataset
It is always good to get a good overview of the dataset. In this case when images are going to be classified a good may to explore the dataset is by looking at the images. If you don't trust the dataset you should go through all images and see if the classification is correct. 

Remember that there is a huge job done preparing and classifying all images. If you are going to do a classification task youself you first need to classify a huge amount of images.

Note that some of the numbers are quite hard to classify even for the human eye. 

In [ ]:
f, ax = plt.subplots(10, 10, figsize=(10, 10))
ax = ax.flatten()
for i in range(len(ax)):
    ax[i].imshow(x_test[i, :, :, 0], cmap='gray')
    ax[i].set_title(y_test[i])
[axi.set_axis_off() for axi in ax.ravel()]
plt.subplots_adjust(bottom=-0.09, wspace=0.03)
plt.show()

### Exercise 
Look at 100 different images

In [ ]:
# Write some code that plots the last 100 images in the training dataset.

In [ ]:
# Scale the dataset to have values between 0 and 1
if np.max(x_train)>1:
    x_train = x_train / 255.0
    x_test = x_test / 255.0
else:
    print('Already scaled once')

### Exercise
From the images above which numbers do you think are hardest and easiest for the model to classify? 

Which numbers do you think gets mixe up by the model?

# Create the conv net
Instead of Densly coupled layers that only cares about each pixel. We try to make a more general network that tries to see more general characteristics of the image. 

In this first example we are using convolutional layers to "look" at 3x3 pixels and then reduce the size of the image with MaxPooling layer.



In [ ]:
# Creating a Sequential Model and adding the layers
input_shape = (28, 28, 1) #The shape of each image
model = Sequential()
model.add(Conv2D(28, kernel_size=(3,3), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
model.add(Dropout(0.1))
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dropout(0.1))
model.add(Dense(10,activation=tf.nn.softmax))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])


### Look at the model
To get an idea of how many parameters we are trying to calculate and to get the model configuration we can use the summary method. 


In [ ]:
model.summary()

### Calculation of parameters
In the first convolution layer there is only 280 parameters to calculate compared to the dens layer with over 600' parameters. In the convolution layer each filter is 3x3 hence 9 parameters is needed to describe the filter function. Additionally we have a bias for each filter then we have 28 filters in the convolutional layer. 
* (3 * 3 + 1) * 28 = 280


Before the dense layer we have flattened the resulting image of the convolutional layer. 
* 13 * 13 * 28 = 4732

Each neuron in the dense layer is connected to every neuron in the output of the previous layer resulting in:
*  (4732 + 1) * 128 = 605824

In the the classification layer there is 10 neurons, one for each class that are connected with the 128 output parameters. There is also one bias parameter for each node. 
* (28 + 1) * 10=290

In [ ]:
# Fit the model to the training dataset
history=model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2, batch_size=256)

In [ ]:
# summarize history for accuracy
f,ax=plt.subplots(1,2,figsize=(12,4))
ax[0].plot(history.history['acc'], label='train')
ax[0].plot(history.history['val_acc'], label='test')
ax[0].set_title('Model accuracy')
ax[0].set_ylabel('accuracy')
ax[0].set_xlabel('epoch')
ax[0].legend()

# summarize history for loss
ax[1].plot(history.history['loss'], label='train')
ax[1].plot(history.history['val_loss'], label='test')
ax[1].set_title('Model loss / cost')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('epoch')
ax[0].legend()
plt.show()

In [ ]:
prediction = model.predict_classes(x_test, verbose=1, batch_size=512)

In [ ]:
class_names= np.unique(y_test)
plot_confusion_matrix(y_test, prediction, classes=class_names, normalize=True,
                      title='Normalized confusion matrix',fig_size=10)
plt.show()
print(classification_report(y_test, prediction, digits = 3))

### Discussion about the confusion matrix
Even though just two epochs was used to calculate classification model. It has come up with a rather good model. A very interesting observation in the confusion matrix above is that the model mixes numbers that are quite closesly looking for example 3 and 5, 7 and 2 and 4 and 9. It is quite easy to understand why and how those numbers can be confusing for the system.

### Investigate the errors
In the cell below all errors are plotted. In a lot of cases we can understand why the model predicted wrong. But in a lot of predictions we can expect the model to do better. 

In [ ]:
plot_errors(x_test, y_test, prediction, 600)

### Creating a slightly better model

In [ ]:
# Creating a Sequential Model and adding the layers
input_shape = (28, 28, 1)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
# Flattening the 2D arrays for fully connected layers
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dropout(0.5))
model.add(Dense(10, activation=tf.nn.softmax))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [ ]:
history=model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=256)
model.evaluate(x_test, y_test)

In [ ]:
# summarize history for accuracy
f,ax=plt.subplots(1,2,figsize=(12,4))
ax[0].plot(history.history['acc'], label='train')
ax[0].plot(history.history['val_acc'], label='test')
ax[0].set_title('Model accuracy')
ax[0].set_ylabel('accuracy')
ax[0].set_xlabel('epoch')
ax[0].legend()

# summarize history for loss
ax[1].plot(history.history['loss'], label='train')
ax[1].plot(history.history['val_loss'], label='test')
ax[1].set_title('Model loss / cost')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('epoch')
ax[0].legend()
plt.show()


# Evaluate the result
One way to evaluate the result is throug the confusion matrix. 
### Exercise
What is the most common error of our classification model?

Compared to the artificial neural network. How does this setup work?

In [ ]:
output = model.predict_classes(x_test, verbose=1, batch_size=512)

In [ ]:
print(classification_report(y_test, output, digits = 3))
class_names= np.unique(y_test)#.astype(str)
plot_confusion_matrix(y_test, prediction, classes=class_names, normalize=True,
                      title='Normalized confusion matrix',fig_size=10)
plt.show()

In [ ]:
plot_errors(x_test, y_test, output, 600)

# Look at the filters/kernels
In the presentation there where a few slides about the filters. In this model we have used 3x3 filters and max poolinglayers.

### Exercise
Look at the filters and try to determine what features are extracted/enhanced by that filter?

In [ ]:
# First plot 
f,ax=plt.subplots(1,11,figsize=(20,2))
p=0
for j in [1,3,5,7,2,11,13,38,17,4]:
    im = x_train[j,:,:,0]
    ax[p].imshow(im,cmap='gray')
    p+=1
for i in range(11):
    ax[i].axis('off')
plt.show()

for i in range(np.shape(model.layers[0].get_weights()[0])[-1]):
    f,ax=plt.subplots(1,11,figsize=(20,2))
    p=0
    for j in [1,3,5,7,2,11,13,38,17,4]:
        im = x_train[j,:,:,0]
        k = model.layers[0].get_weights()[0][:,:,0,i]
        im = ndimage.convolve(im, k)
        im = measure.block_reduce(im, (2,2), np.max)
        
        ax[p].imshow(im,cmap='gray')
        p+=1
    ax[10].imshow(k,cmap='gray')
    for i in range(11):
        ax[i].axis('off')
    plt.show()

In [ ]:
model.summary()

### Data augumentation
A simple trick to increase the dataset is to make small changes to the images for each training cycle. One of those image generators is ImageDataGenerator included in Keras. 

Below is a small simple code example of how to use ImageDataGenerator. Note that we are not allowing vertical or horizontal flip since numbers are orientation dependent. In opposite to faces where we could use horizontal flip.

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=["accuracy"])


# initialize the number of epochs and batch size
EPOCHS = 2
BS = 256
 
# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=5, 
                         zoom_range=0.05, 
                         width_shift_range=0.1, 
                         height_shift_range=0.1, 
                         shear_range=0.1, 
                         horizontal_flip=False, 
                         vertical_flip=False, 
                         fill_mode="nearest")

H = model.fit_generator(aug.flow(x_train, y_train, batch_size=BS),
                        validation_data=(x_test , y_test), steps_per_epoch=len(x_train) // BS,
                        epochs=EPOCHS)

In [ ]:
output = model.predict_classes(x_test, verbose=1, batch_size=512)

In [ ]:
print(classification_report(y_test, output, digits = 3))

plot_confusion_matrix(y_test, output, classes=class_names, normalize=True,
                      title='Normalized confusion matrix',fig_size=10)
plt.show()

In [ ]:
# summarize history for accuracy
f,ax=plt.subplots(1,2,figsize=(12,4))
ax[0].plot(history.history['acc'], label='train')
ax[0].plot(history.history['val_acc'], label='test')
ax[0].set_title('Model accuracy')
ax[0].set_ylabel('accuracy')
ax[0].set_xlabel('epoch')
ax[0].legend()

# summarize history for loss
ax[1].plot(history.history['loss'], label='train')
ax[1].plot(history.history['val_loss'], label='test')
ax[1].set_title('Model loss / cost')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('epoch')
ax[0].legend()
plt.show()


### Exercise
What kind of problems could this deep learning model solve in your organization?